# Topics in Quantitative Finance, Summer 2023 

## Lecture 5: Volatility and volatility-linked derivatives

<br>
<br>

<center>
<font size=5, color=darkblue> Tai-Ho Wang (王 太和)</font>
</center>
<img src="http://mfe.baruch.cuny.edu/wp-content/uploads/2016/04/MFE-Logo.jpg" align = "center" width=450>

$$
\newcommand{\bea}{\begin{eqnarray}}
\newcommand{\eea}{\end{eqnarray}}
\newcommand{\supp}{\mathrm{supp}}
\newcommand{\F}{\mathcal{F} }
\newcommand{\cF}{\mathcal{F} }
\newcommand{\E}{\mathbb{E} }
\newcommand{\Eof}[1]{\mathbb{E}\left[ #1 \right]}
\newcommand{\Etof}[1]{\mathbb{E}_t\left[ #1 \right]}
\def\Cov{{ \mbox{Cov} }}
\def\ES{{ \mbox{ES} }}
\def\Var{{ \mbox{Var} }}
\def\VaR{{ \mbox{VaR} }}
\def\sd{{ \mbox{sd} }}
\def\corr{{ \mbox{corr} }}
\newcommand{\1}{\mathbf{1} }
\newcommand{\p}{\partial}
\newcommand{\PP}{\mathbb{P} }
\newcommand{\Pof}[1]{\mathbb{P}\left[ #1 \right]}
\newcommand{\QQ}{\mathbb{Q} }
\newcommand{\R}{\mathbb{R} }
\newcommand{\DD}{\mathbb{D} }
\newcommand{\HH}{\mathbb{H} }
\newcommand{\spn}{\mathrm{span} }
\newcommand{\cov}{\mathrm{cov} }
\newcommand{\HS}{\mathcal{L}_{\mathrm{HS}} }
\newcommand{\Hess}{\mathrm{Hess} }
\newcommand{\trace}{\mathrm{trace} }
\newcommand{\LL}{\mathcal{L} }
\newcommand{\s}{\mathcal{S} }
\newcommand{\ee}{\mathcal{E} }
\newcommand{\ff}{\mathcal{F} }
\newcommand{\hh}{\mathcal{H} }
\newcommand{\bb}{\mathcal{B} }
\newcommand{\dd}{\mathcal{D} }
\newcommand{\g}{\mathcal{G} }
\newcommand{\half}{\frac{1}{2} }
\newcommand{\T}{\mathcal{T} }
\newcommand{\bit}{\begin{itemize}}
\newcommand{\eit}{\end{itemize}}
\newcommand{\beq}{\begin{equation}}
\newcommand{\eeq}{\end{equation}}
\newcommand{\tr}{\mbox{tr}}
\newcommand{\angl}[1]{\langle #1 \rangle}
$$


## Agenda

- Volatility and its various estimators
    - Historical volatility
    - Implied volatility
    - Realized variance
    - VIX 
- Implied volatility
    - Estimating discount factor and dividend rates by put-call parity
    - GPR fit of implied volatilities
- Volatility indices published by CBOE
- Volatility linked derivatives
    - VIX option
    
- Appendix (Optional)
    - Realized variance from high frequency data
    - The Corsi HAR-RV forecast

## What is volatility?

From the [Wikipage](https://en.wikipedia.org/wiki/Volatility_(finance)):

>In finance, volatility (symbol $\sigma$) is the degree of variation of a trading price series over time as measured by the standard deviation of logarithmic returns.
>
>- Historic volatility measures a time series of past market prices. 
>- Implied volatility looks forward in time, being derived from the market price of a market-traded derivative (in particular, an option). 
>- Realized variance estimates the integrated variance or quadratic variation by using high frequency data.

## Why is volatility important?

From the same [Wikipage](https://en.wikipedia.org/wiki/Volatility_(finance)):

>Investors care about volatility for at least the following reasons:
>
>- The wider the swings in an investment's price, the harder emotionally it is to not worry;
>
>- Price volatility of a trading instrument can define position sizing in a portfolio;
>
>- When certain cash flows from selling a security are needed at a specific future date, higher volatility means a greater chance of a shortfall;
>
>- Higher volatility of returns while saving for retirement results in a wider distribution of possible final portfolio values;
>
>- Higher volatility of return when retired gives withdrawals a larger permanent impact on the portfolio's value;
>
>- Price volatility presents opportunities to buy assets cheaply and sell when overpriced;
>
>- <font color=blue> Portfolio volatility has a negative impact on the compound annual growth rate (CAGR) of that portfolio </font>
>
>- Volatility affects pricing of options, being a parameter of the Black–Scholes model.
>

In today's markets, it is also possible to trade volatility directly, through the use of derivative securities such as options and variance swaps. 

## Volatilities

Volatiltiy of a financial asset in its most prelimanry form is defined as the (conditional) standard deviation of its log return. In practice, there exist various notions of "volatility" that are commonly used including  

- Historical volatility
- Realized and integrated variance/volatility
- Implied volatility 
- Instantaneous volatility 

and methods of inferring these volatilities respectively from

- Daily or high-frequency time series data of the underlying
- Price series of variance swap
- Prices of liquidly traded vanilla options
 

## Historical volatility

Historical volatility uses, say daily, price series to calculate the sample conditional standard deviation of log returns in rolling time windows of a prespecified width.  

##  Example of historical volatility

Now let's calculate the volatility of S&P500 using 25-day rolling time windows. 

One can calculate the volatiltiy series of the input price series $S_t$, $1 \leq t \leq T$, by the following formula

$$
\sigma_t = \sqrt{\frac N{n-2}\sum_{i=1}^{n-1} (r_i - \bar r)^2}, \quad \mbox{ for } n \leq t \leq T,
$$

where 

\begin{eqnarray*}
&& r_i = \ln S_{t + i} - \ln S_{t + i - 1}, \mbox{ for } 1 \leq i \leq n - 1\\
&& \bar r = \frac1{n-1} \sum_{i=1}^{n-1} r_i. 
\end{eqnarray*}

### Note
- $n$ denotes the width (number of days) of rolling window
- $N$ denotes the number of days in a year, thus $\sqrt N$ is the annualizing factor
- The first $n-1$ points in the output volatility series appear as NA, for an obvious reason.

### Exponetially weighted moving average (EWMA)
An alternative to calculate historical volatility is the *exponentially weighted moving average* method. 

$$
\sigma_t = \sqrt{N(1 - \lambda)\sum_{i=1}^{\infty} \lambda^i (r_{t - i} - \bar r)^2}, \quad \mbox{ for } n \leq t \leq T,
$$

for some $\lambda \in (0, 1)$.

## Now let's dirty our hands ...

In [3]:
import numpy as np
from numpy import exp, log, sqrt
import scipy.stats as ss
from scipy.stats import norm
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

ModuleNotFoundError: No module named 'yfinance'

In [2]:
start, end = '2019-01-01', '2022-07-29'
# download SPX from yahoo finance
spx = yf.download('^GSPC', start=start, end=end)

NameError: name 'yf' is not defined

In [ ]:
# brief look at the data
spx.info(), spx.isna().sum()

In [ ]:
# first and last few rows of the data
spx

In [ ]:
# save data, just in case
#spx.to_csv('spx_07252022.csv')

In [ ]:
# load saved data
#spx = pd.read_csv('spx_07252022.csv')
#spx.index = spx['Date']
#spx = spx.drop('Date', axis=1)

In [ ]:
# summary statistics
spx.describe().transpose()

In [ ]:
# plot spx adjusted close
plt.figure(figsize=(9, 6))
spx['Adj Close'].plot()
plt.ylabel('SPX', fontsize=12)
plt.grid();

In [ ]:
# log return of spx 
r = log(spx['Adj Close']).diff()

# historical volatility in this period
vol = r.std()*sqrt(252)
r, r.mean(), vol, type(r)

In [ ]:
# plot log return
plt.figure(figsize=(9, 6))
r.plot(lw=1)
plt.ylabel('SPX log return', fontsize=12)
plt.grid();

In [ ]:
r.iloc[0:10], r.iloc[-10:]

In [ ]:
# n: rwidth of rolling window
# N: annualizing factor
def volatility(data, n=10, N=252):
    data = pd.Series(data)
    vol = [np.nan for i in range(n)]
    for i in range(len(data)-n):
        vol += [data.iloc[i:(i+n)].std()*sqrt(N)]
    return pd.DataFrame({'volatility': vol})

In [ ]:
volatility(r, n=25)

In [ ]:
spx_vol = volatility(r)
spx_vol.index = r.index
plt.figure(figsize=(9, 6))
plt.plot(spx_vol)
plt.ylabel('Volatility', fontsize=12)
plt.title('SPX historical volatility', fontsize=24)
plt.grid();

In [ ]:
# leverage effect
spx_scaled = (spx - spx.mean())/(spx.max() - spx.min())
plt.figure(figsize=(9, 6))
spx_scaled['Adj Close'].plot(color='k', lw=1, label='scaled spx')
plt.ylim([-0.6, 1])
plt.plot(spx_vol, 'b-.', lw=1, label='volatility')
plt.grid()
plt.title('Leverage effect', fontsize=24)
plt.legend();

## Volatility estimation using OHLC

- Parkison
- Garman-Klass
- Rogers-Satchell
- Yang-Zhang


\begin{eqnarray*}
&& \sigma_P^2 = \frac1{4\ln2} \frac1n\sum_{i=1}^n (\ln H_i - \ln L_i)^2 \\
&& \sigma_{GK}^2 = \frac1n \left\{\sum_{i=1}^n \frac12(\ln H_i - \ln L_i)^2 + (2\ln2 - 1)(\ln C_i - \ln O_i)^2 \right\} \\
&& \sigma_{RS}^2 = \frac1n \sum_{i=1}^n u_i(u_i - c_i) + d_i(d_i - c_i) \\
&& \sigma_{YZ}^2 = \sigma_O^2 + w\sigma_C^2 + (1 - w)\sigma_P^2, \quad w = \frac{0.34}{1.34 + \frac{n+1}{n-1}}.
\end{eqnarray*}

- $u_i = \ln H_i - \ln O_i$: daily high weighted by open, 
- $d_i = \ln L_i - \ln O_i$: daily low weighted by open
- $o_i = \ln O_i - \ln C_{i-1}$
- $c_i = \ln C_i - \ln O_i$

We make the calculation of these volatilties into a python `class`. 

In [ ]:
class Volatilities:
    def __init__(self, OHLC, n=10, N=252):
        self.n = n
        self.N = N
        self.OHLC = pd.DataFrame(OHLC)
        self.o = self.OHLC.Open
        self.h = self.OHLC.High
        self.l = self.OHLC.Low
        self.c = self.OHLC.Close
        self.r = log(self.OHLC['Adj Close']).diff()
        self.vols_c = [np.nan for i in range(self.n)] 
        self.vols_p = [np.nan for i in range(self.n)]
        self.vols_gk = [np.nan for i in range(self.n)] 
        self.vols_rs = [np.nan for i in range(self.n)] 
        
        for i in range(len(self.r) - self.n):
            self.vols_c += [self.r.iloc[i:(i+self.n)].std()*sqrt(self.N)]
            self.vols_p += [self.cal_vol_p(self.h.iloc[i:(i+self.n)], self.l.iloc[i:(i+self.n)])*sqrt(self.N)]
            self.vols_gk += [self.cal_vol_gk(self.o.iloc[i:(i+self.n)], self.h.iloc[i:(i+self.n)], self.l.iloc[i:(i+self.n)], self.c.iloc[i:(i+self.n)])*sqrt(self.N)]
            self.vols_rs += [self.cal_vol_rs(self.o.iloc[i:(i+self.n)], self.h.iloc[i:(i+self.n)], self.l.iloc[i:(i+self.n)], self.c.iloc[i:(i+self.n)])*sqrt(self.N)]
        self.vols = pd.DataFrame({'close': self.vols_c, 'parkinson': self.vols_p, 'garman-klass': self.vols_gk, 'rogers-satchell': self.vols_rs})
        self.vols.index = self.OHLC.index
        
    def cal_vol_p(self, H, L):
        return np.sqrt(((log(H) - log(L))**2).mean()/log(2)/4)
    
    def cal_vol_gk(self, O, H, L, C):
        term1 = ((log(O) - log(L))**2).mean()/2
        term2 = (2*log(2) - 1)*((log(C) - log(O))**2).mean()
        return np.sqrt(term1 + term2)
    
    def cal_vol_rs(self, O, H, L, C):
        u, d, c = log(H) - log(O), log(L) - log(O), log(C) - log(O)
        return np.sqrt((u*(u-c)).mean() + (d*(d-c)).mean())

In [ ]:
%%time
spx_vols = Volatilities(spx)

In [ ]:
plt.figure(figsize=(10, 6))
spx_vols.vols['close'].plot(ls='--', label='Close', lw=0.8)
spx_vols.vols['parkinson'].plot(lw=0.8, label='Parkinson')
spx_vols.vols['garman-klass'].plot(lw=0.8, label='Garman-Klass')
spx_vols.vols['rogers-satchell'].plot(lw=0.8, label='Rogers-Satchell')
plt.legend()
plt.grid();

## Implied volatility

"A wrong number to a wrong formula for a correct answer."

- In the Black-Scholes model there is a one-to-one relation between the price of the option and the volatility parameter $\sigma$. The option prices are often quoted by stating this specific volatility, called the *implied volatility*.

- In Black-Scholes world, the volatility is assumed constant. But in reality, options of different strike require different volatilities to match their market prices. This is called the *volatility smile*.

- Most of the work was inspired in modeling the implied volatility.

## Why implied volatility rather than the price itself?

- Price of a call option is decreasing in strike and increasing in time to expiry
- Price of a far out-of-money option is small whereas price of a far in-the-money option carries mostly the intrinsic value
- Statistically speaking, implied volatility is a more standard quantity to infer
- Traders trade options in terms of implied volatilities rather than their prices

## A practical tip for fetching risk free and dividend rates

Q: How to obtain the interest rate $r$ and dividend rate $d$ for the calculation of implied volatility?

A: Put-call parity.

Recall the Put-Call Parity for European options

$$
C - P = Se^{-dT}  - K e^{-rT} = e^{-rT} (F - K),
$$

where $F$ denotes the forward price of the underlying. 
Hence, if we regress $C - P$ against $K$, the (negative) slope gives us the discount factor and the intercept gives the ex-dividend underlying.  

## Example - Implied volatilities of options on SPX

In [ ]:
# import required modules
import datetime
from datetime import datetime as dt
import numpy as np
from numpy import exp, log, sqrt
import scipy.stats as ss
from scipy.stats import norm
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

In [ ]:
# download spx options data from yahoo finance
spx = yf.Ticker('^SPX')
spx_expiries = spx.options
print(spx_expiries)

In [ ]:
# choose an expiry 
idx = 17
today = dt.strftime(dt.now(), '%Y-%m-%d')
day_count = (dt.strptime(spx_expiries[idx], '%Y-%m-%d') - dt.now()).days
print(f'option expiry = {spx_expiries[idx]}, today = {today}')
print(f'There are {day_count} days to expiry')
spx_calls, spx_puts = spx.option_chain(spx_expiries[idx])

In [ ]:
spx_calls.shape, spx_puts.shape

In [ ]:
spx_calls.info()

In [ ]:
spx_calls.isna().sum()

In [ ]:
spx_puts.isna().sum()

In [ ]:
spx_calls

In [ ]:
# clean up data
# remove NA's
spx_calls = spx_calls.drop(['currency', 'contractSize'], axis=1).dropna()
spx_puts = spx_puts.drop(['currency', 'contractSize'], axis=1).dropna()
spx_calls.shape, spx_puts.shape

In [ ]:
spx_calls[spx_calls['lastTradeDate'] > '2022-07']

In [ ]:
# remove data point where either bid = 0 or ask = 0
# remove data that that are not traded recently
spx_calls = spx_calls[(spx_calls['bid'] > 0) & (spx_calls['ask'] > 0)]
#spx_calls[spx_calls['lastTradeDate'] > '2022-07']
spx_calls

In [ ]:
spx_puts = spx_puts[(spx_puts['bid'] > 0) & (spx_puts['ask'] > 0)]
#spx_puts = spx_puts[spx_puts['lastTradeDate'] > '2022-02']
spx_puts

In [ ]:
# save data as csv
#spx_calls.to_csv('spxcall_07252022.csv', index=False)
#spx_puts.to_csv('spxput_07252022.csv', index=False)

In [ ]:
# read saved spx option data
spx_calls = pd.read_csv('spxcall_07252022.csv')
spx_puts = pd.read_csv('spxput_07252022.csv')

###  Create `python` class `OptionAnalytics`

Wrap everything up in a python class.

In [ ]:
class OptionAnalytics:
    def __init__(self, option_chain, expiry, today):
        self.expiry = expiry
        self.today = today
        if not type(today) == datetime.datetime:
            self.today = dt.strptime(today, '%Y-%m-%d')
        self.calls, self.puts = option_chain
        self.ks_c = self.calls['strike']
        self.cs = (self.calls['bid'] + self.calls['ask'])/2 
        self.ks_p = self.puts['strike'] 
        self.ps = (self.puts['bid'] + self.puts['ask'])/2         
        # strikes that are traded for both calls and puts
        self.ks = np.array([])
        for k in self.ks_c:
            if k in np.array(self.ks_p):
                self.ks = np.concatenate([self.ks, [k]])

        self.mids_call = np.array([])
        for k in self.ks:
            self.calls[self.calls['strike'] == k]['bid']
            bid = self.calls[self.calls['strike'] == k]['bid']
            ask = self.calls[self.calls['strike'] == k]['ask']
            self.mids_call = np.concatenate([self.mids_call, np.array((bid + ask)/2)])

        self.mids_put = np.array([])
        for k in self.ks:
            bid = self.puts[self.puts['strike'] == k]['bid']
            ask = self.puts[self.puts['strike'] == k]['ask']
            self.mids_put = np.concatenate([self.mids_put, np.array((bid + ask)/2)])
            
        tmp = self.imp_vols()
        self.ivs, self.s_adj = tmp['imp_vols'], tmp['s_adj']
        
    # put-call parity plot 
    def plot_parity(self):
        plt.figure(figsize=(9, 5))
        plt.plot(self.ks, self.mids_call - self.mids_put, 'r.--')
        plt.ylabel(r'$C - P$', fontsize=12)
        plt.xlabel(r'$K$', fontsize=12)
        plt.title(f'Expiry: {self.expiry}', fontsize=15);
        return None
            
    def plot_arb(self):
        # monotonicity and convexity for option premia vs strikes
        fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)
        axes[0].plot(self.ks_c, self.cs, 'bo--')
        axes[0].set_ylabel('Option mid price', fontsize=12)
        axes[0].set_xlabel('Strike', fontsize=12)
        axes[1].plot(self.ks_p, self.ps, 'ro--')
        axes[1].set_xlabel('Strike', fontsize=12);
        return None
        
    # plot implied vol
    def plot_imp_vols1(self):
        ivs_c = self.calls['impliedVolatility']
        ivs_p = self.puts['impliedVolatility']
        # plot 
        plt.figure(figsize=(10, 6))
        plt.plot(self.ks_p, ivs_p, 'ro--', label='implied vol from puts')
        plt.plot(self.ks_c, ivs_c, 'bo--', label='implied vol from calls')
        plt.title(f'Expiration date: {self.expiry}', fontsize=15)
        plt.xlabel('Strikes', fontsize=12)
        plt.ylabel('Impliied Volatilities', fontsize=12)
        plt.legend();
        return None
        
    # Black-Scholes formula for call
    def bs_call(self, s, K, t, sigma, r=0):
        d1 = (log(s/K) + r*t)/(sigma*sqrt(t)) + sigma*sqrt(t)/2
        d2 = d1 - sigma*sqrt(t)
        return s*norm.cdf(d1) - K*exp(-r*t)*norm.cdf(d2)
    
    # function calculating implied vol by the bisection method
    def bs_impvol_call(self, s0, K, T, C, r=0):
        K = np.array([K])
        n = len(K)
        sigmaL, sigmaH = 1e-10*np.ones(n), 10*np.ones(n)
        CL, CH = self.bs_call(s0, K, T, sigmaL, r), self.bs_call(s0, K, T, sigmaH, r)
        while np.mean(sigmaH - sigmaL) > 1e-10:
            sigma = (sigmaL + sigmaH)/2
            CM = self.bs_call(s0, K, T, sigma, r)
            CL = CL + (CM < C)*(CM - CL)
            sigmaL = sigmaL + (CM < C)*(sigma - sigmaL)
            CH = CH + (CM >= C)*(CM - CH)
            sigmaH = sigmaH + (CM >= C)*(sigma - sigmaH)    
        return sigma[0]
    
    # calculate implied vols
    def imp_vols(self):
        # regress call - put over strike K 
        # apply put-call parity 
        df = {'CP': self.mids_call - self.mids_put, 'Strike': self.ks}
        result = sm.ols(formula='CP ~ Strike', data=df).fit()
        s_adj, pv = result.params[0], -result.params[1]
        ks_pv = self.ks*pv
        days_to_expiry = (dt.strptime(self.expiry, '%Y-%m-%d') - self.today).days
        imp_vols = self.bs_impvol_call(s_adj, ks_pv, days_to_expiry/365, self.mids_call, r=0)        
        return {'imp_vols': imp_vols, 'pv': pv, 's_adj': s_adj}
    
    # plot implied vol
    def plot_imp_vols2(self):
        plt.figure(figsize=(10, 6))
        y = self.ivs[self.ivs>0.001]
        x = self.ks[self.ivs>0.001]
        plt.plot(log(x/self.s_adj), y, 'b.--')
        plt.plot(log(x/self.s_adj), y, 'r.')
        plt.xlabel('logmoneyness', fontsize=12)
        plt.ylabel('implied volatilities', fontsize=12)
        plt.title('Implied volatilities vs Logmoneyness', fontsize=15);
        return None
    
    def __call__(self):
        pass

In [ ]:
expiry = '2022-08-26'
today = '2022-07-25'
# today = dt.now()
spx_opt = OptionAnalytics([spx_calls, spx_puts], expiry, today)
#spx_opt = OptionAnalytics([spx_calls, spx_puts], spx_expiries[idx])

In [ ]:
spx_opt.plot_arb()

In [ ]:
spx_opt.plot_parity()

In [ ]:
spx_opt.imp_vols()

In [ ]:
spx_opt.plot_imp_vols2(), spx_opt.plot_imp_vols1();

## A shorter time to expiry option on SPX

In [ ]:
# choose an expiry 
idx = 5
today = dt.strftime(dt.now(), '%Y-%m-%d')
day_count = (dt.strptime(spx_expiries[idx], '%Y-%m-%d') - dt.now()).days
print(f'option expiry = {spx_expiries[idx]}, today = {today}')
print(f'There are {day_count} days to expiry')
spx_calls1, spx_puts1 = spx.option_chain(spx_expiries[idx])

In [ ]:
# clean data
spx_calls1 = spx_calls1.dropna()
spx_calls1 = spx_calls1[(spx_calls1['bid'] > 0) & (spx_calls1['ask'] > 0)]
spx_puts1 = spx_puts1.dropna()
spx_puts1 = spx_puts1[(spx_puts1['bid'] > 0) & (spx_puts1['ask'] > 0)]

In [ ]:
today = dt.now()
spx_opt1 = OptionAnalytics([spx_calls1, spx_puts1], spx_expiries[idx], today)

In [ ]:
spx_opt1.plot_arb()

In [ ]:
spx_opt1.plot_parity()

In [ ]:
spx_opt1.s_adj, spx_opt1.imp_vols()['pv']

In [ ]:
spx_opt1.plot_imp_vols2(), spx_opt1.plot_imp_vols1();

## GPR fit for SPX implied volatility curve

In [ ]:
# The posterior mean function from GPR
# inputs: 
# m: prior mean function 
# k: prior kernel
# y: observations
# x: indices
# 
# output: the posterior mean function

def pos_mean(m, k, y, x, sigma=0.001):
    n = len(x)
    
    # calculate the covariance matrices
    tmp, _ = np.meshgrid(x, x)
    Sigma_YY = k(tmp, _)
    Sigma_YY = Sigma_YY + sigma**2*np.identity(n)
    
    # determine Sigma_YY_inv(Y - EY) by solving the linear system Sigma_YY x = Y - EY
    Sigma_YY_inv_Y_EY = np.linalg.solve(Sigma_YY, y - m(x))
    
    # return the posterior mean function
    return lambda xx: m(xx) + sum(k(xx, x)*Sigma_YY_inv_Y_EY)


# The posterior kernel from GPR
# inputs: 
# k: prior kernel
# x: indices
# 
# output: the posterior kernel

def pos_kernel(k, x, sigma=0.1):
    n = len(x)
    
    # calculate the covariance matrices
    tmp, _ = np.meshgrid(x, x)
    Sigma_YY = k(tmp, _)
    Sigma_YY = Sigma_YY + sigma**2*np.identity(n)
    
    # return the posterior kernel
    def _(xx, xp):
        # determine Sigma_YY_inv Sigma_Yxp by solving the linear system Sigma_YY x = Sigma_Yxp
        Sigma_YY_inv_Yxp = np.linalg.solve(Sigma_YY, k(xp, x))
        Sigma_xY = k(xx, x)
        return k(xx, xp) - sum(Sigma_xY*Sigma_YY_inv_Yxp)
        
    return _ 

In [ ]:
imp_vols = spx_opt.ivs
logmnyns = np.log(spx_opt.ks/spx_opt.s_adj)

# set hyperparameters by guessing
A, l, sigma = 0.1, 0.1, 0.01

# prior mean function
# set prior mean as sample mean of implied vols
iv_mean = imp_vols.mean()
mpr = lambda x: iv_mean

# prior kernel
k = lambda x, y: A*np.exp(-np.abs(x-y)**2/2/l**2)

# indices and observations
x_is = logmnyns
n = len(x_is)
y_is = imp_vols

# posterior mean function for implied vol
mpo_iv = pos_mean(mpr, k, y_is, x_is, sigma=sigma)
mpo_iv = np.vectorize(mpo_iv)
mpo_iv(x_is)


# fitted values
iv_hat = mpo_iv(x_is)
pd.DataFrame(y_is, iv_hat)

In [ ]:
# plot 
plt.figure(figsize=(9, 6))
plt.plot(x_is, y_is, 'bo', label='Market data')
plt.xlabel('logmoneyness')
plt.ylabel('implied vol')
x = np.linspace(x_is.min(), x_is.max(), 200)
plt.plot(x, mpo_iv(x), 'r--', label='GPR fit')
plt.legend();

In [ ]:
# analysis on absolute errors 
abs_errs = np.abs(iv_hat - y_is)
plt.plot(x_is, abs_errs, 'bo')
plt.xlabel('logmoneyness', fontsize=12) 
plt.ylabel('errors', fontsize=12)
pd.DataFrame(abs_errs).describe().transpose()

### Fine tune hyperparameters by MLE

In [ ]:
# estimate the hyperparameters by MLE

# objective function
def obj(x, y):
    def _(params):
        A, l, sigma = params
        k = lambda u, v: A*np.exp(-np.abs(u-v)**2/2/l**2)
        n = len(x)
        # calculate the covariance matrices
        tmp, _ = np.meshgrid(x, x)
        Sigma_YY = k(tmp, _)
        Sigma_YY = Sigma_YY + sigma**2*np.identity(n)
        Sigma_YY_inv_y = np.linalg.solve(Sigma_YY, y)        
        return np.log(np.linalg.det(Sigma_YY)) + sum(y*Sigma_YY_inv_y) #this is in fact negative log likelihood
    return _

In [ ]:
from scipy.optimize import minimize

In [ ]:
%%time
# minimize objective function
print(obj(x_is, y_is)([0.1, 0.1, 0.1]))
pars = [0.1, 0.1, 0.1]
res = minimize(obj(x_is, y_is), pars, method='nelder-mead', 
               options={'xatol': 1e-8, 'disp': True}) 
hyparams = res.x
hyparams

In [ ]:
# hyperparameters estimated from MLE
A, l, sigma = hyparams

# prior mean function
# set prior mean as sample mean of implied vols
iv_mean = imp_vols.mean()
mpr = lambda x: iv_mean

# prior kernel
k = lambda x, y: A*np.exp(-np.abs(x-y)**2/2/l**2)

# indices and observations
x_is = logmnyns
n = len(x_is)
y_is = imp_vols

# posterior mean function for implied vol
mpo_iv = pos_mean(mpr, k, y_is, x_is, sigma=sigma)
mpo_iv = np.vectorize(mpo_iv)
#mpo_iv(x_is)

In [ ]:
# plot 
plt.figure(figsize=(9, 6))
plt.plot(x_is, y_is, 'bo', label='Market data')
plt.xlabel('logmoneyness')
plt.ylabel('implied vol')
x = np.linspace(x_is.min(), x_is.max(), 200)
plt.plot(x, mpo_iv(x), 'r--', label='GPR fit')
plt.legend();

In [ ]:
# analysis on absolute errors 
abs_errs = np.abs(iv_hat - y_is)
plt.plot(x_is, abs_errs, 'bo')
plt.xlabel('logmoneyness', fontsize=12) 
plt.ylabel('errors', fontsize=12)
pd.DataFrame(abs_errs).describe().transpose()

## What is VIX?

Quote from [this page](https://www.investopedia.com/terms/v/vix.asp) in Investopedia:

>Created by the Chicago Board Options Exchange (CBOE), the Volatility Index, or VIX, is a real-time market index that represents the market's expectation of 30-day forward-looking volatility. Derived from the price inputs of the S&P 500 index options, it provides a measure of market risk and investors' sentiments. It is also known by other names like "Fear Gauge" or "Fear Index." Investors, research analysts and portfolio managers look to VIX values as a way to measure market risk, fear and stress before they take investment decisions. 

>Introduced in 1993, the Volatility Index (VIX) was initially a weighted measure of the implied volatility (IV) of eight S&P 100 at-the-money put and call options. Ten years later, in 2004, it expanded to use options based on a broader index, the S&P 500. This expansion allows for a more accurate view of investors' expectations on future market volatility. VIX values higher than 30 are usually associated with a significant amount of volatility as a result of investor fear or uncertainty. Values below 15 ordinarily correspond to less stressful, or even complacent, times in the markets.


- Originally, the VIX computation was designed to mimic the implied volatility of an at-the-money 1 month option on the OEX index. It did this by averaging volatilities from 8 options (puts and calls from the closest to ATM strikes in the nearest and next to nearest months).

- The CBOE changed the VIX computation: “CBOE is changing VIX to provide a more precise and robust measure of expected market volatility and to create a viable underlying index for tradable volatility products.”

- CBOE listed futures on the VIX in 2004.

## Volatility indices published by CBOE

In addition to VIX, other volatility indices published by [CBOE](http://www.cboe.com/products/vix-index-volatility/volatility-on-stock-indexes/cboe-s-p-500-9-day-vol-index-vix9d) include

- VIX9D
- VIX3M
- VIX6M
- VOX
- VXD: Dow Jones index volatility
- RVX
- VXN
- VVIX: VIX of VIX.

### Note
More volatility indices published by CBOE can be found in [this link](http://www.cboe.com/Volatility).

## "Formula of finanical engineering"

Notice that all payoffs we saw before can be expressed as a combination of payoffs from calls and puts, even the underlying itself since it can be regarded as a call struck at zero. A natural question to ask is, to what extent, can a given payoff function be represented as a combination of calls and puts?

The answer is surprisingly "all the payoffs"! The following formula shows how. 

Let $\varphi$ be a payoff function, we have

$$
\varphi(s) = \varphi(f) + \varphi'(f)(s - f) + \int_f^\infty (s - k)^+ \varphi''(k) dk + \int_0^f (k - s)^+ \varphi''(k)dk.
$$

Let $\delta$ denote the Dirac delta function and $\theta$ the Heaviside function. Note that heuristically $\theta' = \delta$, i.e., the Dirac delta can be regarded as the derivative of the Heaviside function. 

The payoff $\varphi(s)$ at time $T$ can be written as 

\begin{eqnarray*}
\varphi(s) &=& \int_0^\infty \varphi(k) \delta(s - k)\,dk \\
&=& \int_0^f \varphi(k) \delta(s - k) dk + \int_f^\infty \varphi(k) \delta(s - k) dk \\
&=& \varphi(f) - \int_0^f \varphi'(k) \theta(k - s) dk + \int_f^\infty \varphi'(k)\theta(s - k) dk \\
&=& \varphi(f) + \varphi'(f) (s - f) + \int_0^f \varphi''(k) (k - s)^+ dk + \int_f^\infty \varphi''(k) (s - k)^+ dk.
\end{eqnarray*}

Thus, 

$$
\varphi(S_T) = \varphi(f) + \varphi'(f) (S_T - f) + \int_0^f \varphi''(k) (k - S_T)^+ dk + \int_f^\infty \varphi''(k) (S_T - k)^+ dk.
$$

With $f = \Eof{S_T}$ and taking expectation on both sides, we end up 
\begin{eqnarray*}
\Eof{\varphi(S_T)} &=&
% \int_0^f\,\varphi''(K)\,(K-S_T)^+\,dK + \int_F^\infty\,g''(k)\,(S_T - k)^+\,dK \\
% &&+ g(F) - g'(F)\,\left[ (F-S_T)^+ - (S_T-F)^+ \right] \\
% &=&
 \varphi(f) + \int_0^f \, \varphi''(k) \, P(k) \,dk + \int_f^\infty\, \varphi''(k)\, C(k) \,dk.
\end{eqnarray*}

- The price of any European style contingent claim can be expressed in terms of strips of out-of-money European options. 


## Example - log contract

Consider the log contract $\varphi(s) = \log s$. Since $\varphi'(s) = \frac1s$ and $\varphi''(s) = -\frac1{s^2}$, we obtain

$$
\log s = \log f + \frac{s - f}{f} - \int_0^f \frac{(k - s)^+}{k^2} dk - \int_f^\infty \frac{(s - k)^+}{k^2} dk. 
$$

Thus, 

$$
\Eof{\log S_T} = \log f - \int_0^f \, \frac{P(k)}{k^2} \,dk - \int_f^\infty\, \frac{C(k)}{k^2} \,dk.
$$


On the other hand, assume $S_t$ satisfies the SDE under risk neutral probability with zero interest and dividend rates

$$
\frac{dS_t}{S_t} = \sigma_t dW_t,
$$

by applying Ito's formula to $\log S_t$, we obtain

\begin{eqnarray*}
\log S_T = \log S_0 + \int_0^T \sigma_t dW_t - \frac12 \int_0^T \sigma_t^2 dt.
\end{eqnarray*}

It follows by taking expectation on both sides that 

$$
\Eof{\log S_T} = \log S_0 - \frac12 \Eof{\int_0^T \sigma_t^2 dt}
$$

Compare the two identities we obtain

$$
\frac1T \Eof{\int_0^T \sigma_t^2 dt} = \frac2T \int_0^f \, \frac{P(k)}{k^2} \,dk + \frac2T \int_f^\infty\, \frac{C(k)}{k^2} \,dk.
$$


### Note
- Modulo the diffusion process assumption on the underlying, the last relationship is model-free.  
- VIX is calculated based on this formula with $T$ equal to a month.  


## How is VIX calculated?

VIX definition in the CBOE white paper:

$$VIX^2=\frac{2}{T}\,\sum_i\,\frac{\Delta K_i}{K_i^2}\,
Q_i(K_i)\,-\,\frac{1}{T}\,\left[\frac{F}{K_0}-1\right]^2
$$

where $Q_i$ is the price of the out-of-the-money option with strike
$K_i$ and $K_0$ is the highest strike below the forward price $F$. $T$ is one month.


### Download VIX data from `yfinance`

In [ ]:
start = '2007-01-01'
end = '2022-07-29'
vix = yf.download('^VIX', start=start, end=end)
vix = vix.drop('Volume', axis=1)
vix

In [ ]:
plt.figure(figsize=(9, 6))
vix['Adj Close'].plot(label='VIX')
#plt.plot(spx_vol*100, 'r-.', label='Historical Volatility')
plt.grid()
plt.legend();

In [ ]:
# plot log(vix)
plt.figure(figsize=(9, 6))
log(vix)['Adj Close'].plot(label='log ViX')
plt.grid()
plt.legend();

## Volatility derivatives

- variance swap
- volatility swap
- VIX futures
- VIX options

## Variance swap

Quote from the [Wikipage](https://en.wikipedia.org/wiki/Variance_swap):

> A variance swap is an over-the-counter financial derivative that allows one to speculate on or hedge risks associated with the magnitude of movement, i.e. volatility, of some underlying product, like an <font color=blue> exchange rate </font>, <font color=blue>interest rate</font>, or stock index.

> One leg of the swap will pay an amount based upon the realized variance of the price changes of the underlying product. Conventionally, these price changes will be daily log returns, based upon the most commonly used closing price. The other leg of the swap will pay a fixed amount, which is the strike, quoted at the deal's inception. Thus the net payoff to the counterparties will be the difference between these two and will be settled in cash at the expiration of the deal, though some cash payments will likely be made along the way by one or the other counterparty to maintain agreed upon margin. 

In summary, a variance swap is a forward contract on realized (annualized) variance whose payoff function ideally is given by  

$$
N \times \left(\frac1T \int_0^T \sigma_t^2 dt - K \right)
$$

where $K$ is the strike and $N$ denotes the notional.


## Volatility swap

Quote from the [Wikipage](https://en.wikipedia.org/wiki/Volatility_swap):

>In finance, a volatility swap is a forward contract on the future realised volatility of a given underlying asset. Volatility swaps allow investors to trade the volatility of an asset directly, much as they would trade a price index.

>The underlying is usually a foreign exchange (FX) rate (very liquid market) but could be as well a single name equity or index. However, the variance swap is preferred in the equity market because it can be replicated with a linear combination of options and a dynamic position in futures.

>Unlike a stock option, whose volatility exposure is contaminated by its stock price dependence, these swaps provide pure exposure to volatility alone. This is truly the case only for forward starting volatility swaps. However, once the swap has its asset fixings its mark-to-market value also depends on the current asset price. One can use these instruments to speculate on future volatility levels, to trade the spread between realized and implied volatility, or to hedge the volatility exposure of other positions or businesses. 


In summary, a volatility swap is a forward contract on realized (annualized) volatility whose payoff function ideally is given by  

$$
N \times \left(\sqrt{\frac1T \int_0^T \sigma_t^2 dt} - K \right)
$$

where $K$ is the strike and $N$ denotes the notional.

## Calculating volatility swap fair strike from MGF

By applying the following formula

$$
\sqrt x = \frac1{2\sqrt\pi}\int_0^\infty s^{-\frac32}\left(1 - e^{-xs}\right) ds,
$$

we obtain  

$$
\Eof{\sqrt{\frac1T \int_0^T \sigma_t^2 dt}} = \frac1{2\sqrt\pi}\int_0^\infty s^{-\frac32}\left(1 - \Eof{e^{-xs}}\right) ds
$$

where apparently,

$$
x = \frac1T \int_0^T \sigma_t^2 dt
$$

and $\Eof{e^{-xs}}$ the moment generating function for realized variance. 

## VIX futures

According to the [VIX page](https://cfe.cboe.com/cfe-products/vx-cboe-volatility-index-vix-futures) on CBOE,

>Introduced in 2004 on Cboe Futures Exchange (CFE), VIX futures provide market participants with the ability to trade a liquid volatility product based on the VIX Index methodology. VIX futures reflect the market's estimate of the value of the VIX Index on various expiration dates in the future. VIX futures provide market participants with a variety of opportunities to implement their view using volatility trading strategies, including risk management, alpha generation and portfolio diversification.

## VIX option

Quote from [this page](https://www.investopedia.com/terms/v/vixoption.asp) in Invstopeida 

>A VIX option is a non-equity index option that uses the CBOE Volatility Index as its underlying asset. Call and put VIX options are both available. The call options hedge portfolios against a sudden market decline, and put options hedge against a rapid reversal of short positions on the S&P 500 index. These options thus allow traders and investors to speculate on future moves in volatility.

## A look at the VIX option chain from Yahoo Finance
Visit this [Yahoo Finance link for VIX option chain](https://finance.yahoo.com/quote/%5EVIX/options/).

In [ ]:
vix = yf.Ticker('^VIX')
vix_expiries = vix.options
print(vix_expiries)

In [ ]:
# choose an expiry 
idx = 4
today = dt.strftime(dt.now(), '%Y-%m-%d')
day_count = (dt.strptime(vix_expiries[idx], '%Y-%m-%d') - dt.now()).days
print(f'option expiry = {vix_expiries[idx]}, today = {today}')
print(f'There are {day_count} days to expiry')
vix_calls, vix_puts = vix.option_chain(vix_expiries[idx])

In [ ]:
# clean data
vix_calls = vix_calls.drop(['currency', 'contractSize'], axis = 1).dropna()
vix_calls = vix_calls[(vix_calls['bid'] > 0) & (vix_calls['ask'] > 0)]
vix_calls

In [ ]:
vix_puts = vix_puts.drop(['currency', 'contractSize'], axis = 1).dropna()
vix_puts = vix_puts[(vix_puts['bid'] > 0) & (vix_puts['ask'] > 0)]
vix_puts

In [ ]:
# save data
#vix_calls.to_csv('vixcall_07252022.csv', index=False)
#vix_puts.to_csv('vixput_07252022.csv', index=False)

In [ ]:
# read in saved data
vix_calls = pd.read_csv('vixcall_07252022.csv')
vix_puts = pd.read_csv('vixput_07252022.csv')

In [ ]:
expiry, today = '2022-08-24', '2022-07-25'
vix_opt = OptionAnalytics([vix_calls, vix_puts], expiry, today)
#vix_opt = OptionAnalytics([vix_calls, vix_puts], vix_expiries[idx], dt.now())

In [ ]:
vix_opt.plot_arb()

In [ ]:
vix_opt.plot_parity()

In [ ]:
vix_opt.plot_imp_vols2(), vix_opt.plot_imp_vols1();

## GPR fit for VIX option implied volatility curve

In [ ]:
imp_vols = vix_opt.ivs[1:]
#imp_vols = vix_opt.ivs
logmnyns = np.log(vix_opt.ks[1:]/vix_opt.s_adj)

# set hyperparameters by guessing
sigma, A, l = 0.05, 0.1, 0.1

# prior mean function
# set prior mean as sample mean of implied vols
iv_mean = imp_vols.mean()
mpr = lambda x: iv_mean

# prior kernel
k = lambda x, y: A*np.exp(-np.abs(x-y)**2/2/l**2)

# indices and observations
x_is = logmnyns
n = len(x_is)
y_is = imp_vols

# posterior mean function for implied vol
mpo_iv = pos_mean(mpr, k, y_is, x_is, sigma=sigma)
mpo_iv = np.vectorize(mpo_iv)
mpo_iv(x_is)


# fitted values
iv_hat = mpo_iv(x_is)
pd.DataFrame(y_is, iv_hat)

In [ ]:
# plot 
plt.figure(figsize=(9, 6))
plt.plot(x_is, y_is, 'bo', label='Market data')
plt.xlabel('logmoneyness')
plt.ylabel('implied vol')
x = np.linspace(x_is.min(), x_is.max(), 200)
plt.plot(x, mpo_iv(x), 'r--', label='GPR fit')
plt.legend();

In [ ]:
# analysis on absolute errors 
abs_errs = np.abs(iv_hat - y_is)
plt.plot(x_is, abs_errs, 'bo')
plt.xlabel('logmoneyness', fontsize=12) 
plt.ylabel('errors', fontsize=12)
pd.DataFrame(abs_errs).describe().transpose()

## Appendix (Optional)

## Volatility estimation using high frequency data

- Market microstructure noise may contaminate the data in high frequency, resulting in inconsistency of estimators. 

## Realized variance

The following estimator is called the *Realized Variance (RV)* estimator

$$
\sum_{i=1}^n \, \left(Y_{t_i} - Y_{t_{i-1}} \right)^2
= \sum_{i=1}^n \, \left( \Delta Y_{t_i} \right)^2, 
$$ 

where $Y_t = \log S_t$ and $S_t$ is the price series of the asset under consideration.


### Technical notes

- Realized variance and realized covariance <br>
    Given a partition $\Pi = \{0 = t_1 < \cdots < t_n=T\}$ of the interval $[0, T]$, the realized variance $[X]_T^\Pi$ of the process $X_t$ sampled at $\Pi$ is defined by
$$
[X]_T^\Pi = \sum_{i=1}^n |X_{t_i} - X_{t_{i-1}}|^2.
$$
    Similiarly, the realized covariance between $X_t$ and $Y_t$ sampled at $\Pi$ is given by 
$$
[X, Y]_T^\Pi = \sum_{i=1}^n (X_{t_i} - X_{t_{i-1}})(Y_{t_i} - Y_{t_{i-1}})
$$

- Quadratic variation (integrated variance) and covariation <br>
    The quadratic variation of $X$ is defined by the limit
    $$
    \angl{X}_t = \lim_{\|\Pi_n\| \to 0} [X]_T^{\Pi_n} 
    $$
    provided the limit exists. $\Pi_n$ denotes a sequence of partitions of the interval $[0, T]$ such that $\|\Pi_n\| \to 0$ as $n \to \infty$, where $\|\Pi_n\|$ denotes the mesh of the partition $\Pi_n$. 

    Likewise, the covariation between and $X$ and $Y$ is defined by the limit
    $$
    \angl{X, Y}_t = \lim_{\|\Pi_n\| \to 0} [X, Y]_T^{\Pi_n}.
    $$



### Assumption
The log price $X_t$ follows the Ito process

$$
dX_t = \mu_t dt + \sigma_t dW_t,
$$

where $W_t$ is a Brownian motion. Under the assumption,
$\angl{X}_t = \int_0^t \sigma_\tau^2 d\tau$.



## Integrated variance or quadratic variation

Given a set of tick data, how can we measure the, say daily, variance? <br>
A possibility is to estimate the *integrated variance*, also known as the *quadratic variation* in the theory of semimartingale. We shall use both terms interchangeably hereafter. 

Recall that the *quadratic variation* $\angl{X}_t$ of the continuous stochastic process $X_t$ is defined by 

$$
\angl{X}_T:= \lim_{\|\Pi_n\| \to 0} \sum_{{t_i} \in \Pi_n} |\Delta X_{t_i}|^2
$$

provided the limit exist (in probability). 

Thus, the goal is to estimate the quadratic variation of the efficient log price from the transacted log price, i.e., tick data. However, the subtlety is that efficient price is not directly observable and is contaminated by market microstructure noises.

#### Note
- If the process $X$ has jumps, the quadratic variation $\angl{X}$ becomes

    $$
    \angl{X}_t = \angl{X^c}_t + \sum_{0 < s \leq t} |\Delta X_s|^2,
    $$

    where $X^c$ denotes the continuous part of $X$ and $\Delta X_s := X_s - X_{s^-}$ is the jump size at time $s$. In this case, the integrated variance usually refers to $\angl{X^c}$, i.e., the quadratic variation of the continuous part. 

- We shall always assume $X$ is a continuous process, thus no jumps, in the sequel. 

### Microstructure noise

In the limit of very high sampling frequency, RV picks up mainly the market
microstructure noise. To see this, suppose that the observed price $Y_t$
is given by

$$Y_t =X_t +\epsilon_t,$$

where $X_t$ is the value of the
underlying (log-)price process of interest and $\epsilon_t$ is a random
market microstructure-related noise term, assumed independent of $X_t$. Suppose we sample the price
series $n+1$ times (so that there are $n$ price changes) at $\Pi=\{0=t_0 < \cdots < t_n = T\}$ in the time interval $[0, T]$. 

Note that, conditioned on $\cF_T^X$, the conditional expectation of the realized variance of transacted (log) price satisties

\begin{eqnarray*}
\Eof{[Y]_T^\Pi |\cF_T^X} &:=& \sum_{i=1}^n \Eof{(\Delta Y_{t_i})^2|\cF_T^X} \\
&=& \sum_{i=1}^n \, (\Delta X_{t_i})^2 + 2 \sum_{i=1}^n \Delta X_{t_i} \Eof{\Delta \epsilon_{t_i}|\cF_T^X} + \sum_{i=1}^n \Eof{(\Delta \epsilon_{t_i})^2|\cF_T^X} \\
&=& [X]_T + 2 n \, \mbox{var}[\epsilon] \\
&\approx& \angl{X}_T + 2 n \, \mbox{var}[\epsilon].
\end{eqnarray*}

#### Note
The difference between $[X]_T$ and $\angl{X}_T$ is referred to as the *discretization error*, which is usually controled by the integrated quarticity $\int_0^T \sigma_t^4 dt$. 

### Asymptotic result

A more detailed, but more technical, asymptotic analysis shown in [Zhang, Mykland and Aït-Sahalia]<sup id="cite_ref-ZMA" class="reference"><a href="#cite_note-ZMA">[9]</a></sup> yields that as $n \to \infty$

$$
[Y]^{\Pi}_T \mathop{\approx}^{\mathcal L} \angl{X}_T + 2 \, n \, \mbox{var}[\epsilon] + \sqrt{ 4 n \Eof{\epsilon^4} + \frac{2T}{n} \int_0^T \sigma_t^4 dt} \; Z,
$$

where $Z \sim N(0,1)$.

#### Note
- The naive RV estimator $[Y]_T^\Pi$ is biased by the variance of market microstructure noise $\epsilon$. The biasedness increases as the sampling frequency $n$ increases.  
- We see that as $n\to\infty$, the naive RV estimator $[Y]_T^\Pi$ picks up mainly the microstructure noise.

### The conventional solution

-   The conventional solution is to sample at most every five minutes or
    so.

    -  For high frequency data, sampling only every 5 minutes usually corresponds to throwing out more than 99% of the points!
        
-   To quote [Zhang, Mykland and Aït-Sahalia]<sup id="cite_ref-ZMA" class="reference"><a href="#cite_note-ZMA">[9]</a></sup>, “It is difficult to accept
    that throwing away data, especially in such quantities, can be an
    optimal solution.”

-   From a more practical perspective, if we believe that volatility is
    time-varying, it makes sense to try and measure it from recent data
    over the last few minutes rather than from a whole day of trading.

### Subsampling

Let $\Pi^{(k)} = \{0 \leq t_0^{(k)} < \cdots < t_{n_k}^{(k)}\leq T\}$, for $1 \leq k \leq K$, be a collection of nonoverlapping subsampling times in $\Pi$. That is, 

$$
\bigcup_{k=1}^K \Pi^{(k)} = \Pi \quad \mbox{ and } \quad \Pi^{(k)} \cap \Pi^{(\ell)} = \emptyset \; \mbox{for } k \neq \ell.
$$

A typical example that we shall be using in the following is by sampling every $K$ ticks from the $k$th tick on. That is, 

\begin{eqnarray*}
&& \Pi^{(1)} = \{t_1 < t_{1+K} < t_{1 + 2K} < \cdots < t_{1 + n_1 K} \leq T \}, \\
&& \Pi^{(2)} = \{t_2 < t_{2+K} < t_{2 + 2K}< \cdots < t_{2 + n_2 K} \leq T \}, \\
&& \qquad \vdots \\
&& \Pi^{(K)} = \{t_0 < t_K < t_{2K}  < \cdots < t_{n_K K} \leq  T \}.
\end{eqnarray*}

We denote by $[Y]_T^{\Pi^{(k)}}$ the RV estimate of $Y$ using the subsamples that are sampled from the sampling times in $\Pi^{(k)}$, for $1 \leq k \leq K$.

By the same token, we have the following asymptotics for each subsample $k \in \{1, \cdots, K\}$

$$
[Y]^{\Pi^{(k)}}_T \mathop{\approx}^{\mathcal L} \angl{X}_T + 2 \, n_k \, \mbox{var}[\epsilon] + \sqrt{ 4 n_k \Eof{\epsilon^4} + \frac{2T}{n_k} \int_0^T \sigma_t^4 dt} \; Z_k
$$

where $Z_k \sim N(0,1)$.

### Boosting RV estimator

We can boost the RV estimator by averaging over the "weak learners" $[Y]_T^{\Pi^{(k)}}$

\begin{eqnarray*}
[Y]_T^{avg} &=& \frac{1}{K} \, \sum_{k=1}^K \, [Y]_T^{\Pi^{(k)}} \\
&\approx& \angl{X}_T + 2 \, \bar n_K \, \mbox{var}[\epsilon] + \sqrt{ 4 \frac{\bar n_K}K \Eof{\epsilon^4} + \frac{4T}{3 \bar n_K} \int_0^T \sigma_t^4 dt} \; Z
\end{eqnarray*}
    
where $\bar n_K := \frac1K \sum_k n_k $ is the average number of ticks in each subsample, roughly equal to $\frac nK$.


#### Note
- Boosting reduces biasedness and variance by a factor of $K$, but is unable to completely remove the biasedness.
- The optimal average subsample size $\bar n^*$ is given by
$$
\bar n^* = \sqrt[3]{\frac T{6\mbox{var}^2[\epsilon]} \int_0^T \sigma_t^4 dt}.
$$
    Thus, the whole sample set is splitted into roughly $K^* \approx \frac n{\bar n^*}$ sets of subsamples.

### The ZMA estimator

Recall that 

\begin{eqnarray*}
&& [Y]^\Pi_T \approx \angl{X}_T + n \, \mbox{var}[\epsilon], \\
&& [Y]^{avg}_T \approx \angl{X}_T + \bar n_K \mbox{var}[\epsilon].
\end{eqnarray*}

We can eliminate bias by forming

\begin{eqnarray*}
\frac1{\bar n_K}[Y]_T^{avg} - \frac1n \, [Y]_T^\Pi \approx \left( \frac1{\bar n_K} - \frac1n \right) \angl{X}_T.
\end{eqnarray*}

Thus we obtain the [Zhang, Mykland and Aït-Sahalia]<sup id="cite_ref-ZMA" class="reference"><a href="#cite_note-ZMA">[9]</a></sup> (ZMA) bias-corrected
estimator of $\angl{X}_T$:

$$
[Y]_T^{ZMA} := \frac{1}{n - \bar n_K} \, \left\{n \, [Y]_T^{avg} - \bar n_K \, [Y]^\Pi_T \right\}.
$$

Moreover, we have the asymptotic behavior for $[Y]_T^{ZMA}$ as

$$
[Y]_T^{ZMA} \approx \angl{X}_T + \frac1{\sqrt[6]n}\sqrt{\frac8{c^2}\mbox{var}^2[\epsilon] + c \frac{4T}3 \int_0^T \sigma_t^4 dt} \; Z
$$

where $Z \sim N(0,1)$. The optimal constant $c^*$ is given by

$$
c^* = \left(\frac T{12 \, \mbox{var}^2[\epsilon]} \int_0^T \sigma_t^4 dt \right)^{-\frac13}.
$$

#### Note
In the original paper [Zhang, Mykland and Aït-Sahalia]<sup id="cite_ref-ZMA" class="reference"><a href="#cite_note-ZMA">[9]</a></sup>, the authors suggested the estimator as $[Y]_T^{avg} - \frac{\bar n_K}{n}[Y]_T^\Pi$, whereas the estimator $[Y]_T^{ZMA}$ obtained above is referred to as the *small-sample adjustment* in the paper. 

### The Zhou estimator

Define

$$
\begin{eqnarray*}
[Y]_T^{\Pi, Z}: &=& \sum_{i=1}^n \, (\Delta Y_{t_i})^2
+ \sum_{i=2}^n \, \Delta Y_{t_i} \, \Delta Y_{t_{i-1}} + \sum_{i=1}^{n-1} \, \Delta Y_{t_i} \Delta Y_{t_{i+1}} \\
&=& \sum_{i=1}^n \,(Y_{t_i} - Y_{t_{i-1}})(Y_{t_{i+1}} - Y_{t_{i-2}}).
\end{eqnarray*}
$$
Thus, under the assumption $Y = X + \epsilon$ of serially uncorrelated
noise independent of returns $X$, we obtain
$\mathbb{E}\left[[Y]^{\Pi, Z}\right] = \Eof{[X]_T}$. 

By further assume $X_t = \sigma W_{\tau(t)}$ (a time-changed Brownian motion) for some Brownian motion $W$ and deterministic increasing function $\tau(\cdot)$, since 

$$
\Eof{(\Delta X_{t_i})^2} = \sigma^2 \Eof{\left\{ W_{\tau(t_i)} - W_{\tau(t_{i-1})} \right\}^2} = \sigma^2 \{\tau(t_i) - \tau(t_{i-1})\},
$$

we have

$$
\mathbb{E}\left[[Y]^{\Pi, Z}\right] = \sigma^2 \{\tau(T) - \tau(0) \} = \angl{X}_T.
$$

In other words, in this case $[Y]^{\Pi, Z}$ is an unbiased estimator of $\angl{X}_T$.

## Realized library

Realized Library at Oxford-Man Institute of Quantitative Finance publishes daily realized variances/volatilities for various indices. 

[https://realized.oxford-man.ox.ac.uk/](https://realized.oxford-man.ox.ac.uk/)

#### Note
<font color=blue>Unfortunately, the Realized Library has ceased providing the service.</font> 

## SPX realized variance

In [ ]:
from datetime import datetime as dt

In [ ]:
df = pd.read_csv('OxfordManRealizedVolatilityIndices.csv', index_col=0, header=2)
rv1 = pd.DataFrame(index=df.index)
for col in df.columns:
    if col[-3:] == '.rk':
        rv1[col] = df[col]

# convert index into datetime
rv1.index = [dt.strptime(str(date), "%Y%m%d") for date in rv1.index.values]

In [ ]:
df

In [ ]:
rv1

In [ ]:
# plot spx realized variance
# spx2.rk contains the rv calcuated by the realized kernel within 5-min bins
spx_rv = pd.DataFrame(rv1['SPX2.rk'])
spx_rv.plot(color='red', grid=True, title='SPX realized variance',
         figsize=(12, 6), ylim=(0,0.003));

## The Corsi HAR-RV forecast

-   The Corsi HAR-RV model implements a regression to fit the parameters.

-   This model can be regarded as a poor man’s version of a long memory
    model such as ARFIMA.

    -   True long-memory models such as ARFIMA are notoriously hard to
        fit.

-   HAR-RV can also be considered an intelligent alternative to GARCH.

-   The model boils down to the regression

    $$RV_{t,t+h} = \beta_0 + \beta_D\,RV_t + \beta_W\,RV_{t-5,t} + \beta_M\,RV_{t-22,t} + \epsilon_{t,t+h}.$$
    
    In words, the RV forecast for $h$ days from now is a linear
    combination of the current realized variance and (aggregate) RV
    estimates for the last week and the last month.
    
### Note
- $RV$ denotes the logarithm of realized variance.

In [ ]:
import statsmodels.formula.api as sm

In [ ]:
# take h = 1 in the HAR-RV model
# y = RV_{t+h}
# rv1 = RV_t
# rv5 = RV_{t-5}
# rv22 = RV_{t-22}
spx_rv = spx_rv.dropna()
spx1 = np.array(spx_rv)
y = spx1[22:]
rv1 = spx1[21:-1]
rv5 = np.array(pd.DataFrame(spx1[17:]).rolling(5).mean()[5-1:-1])
rv22 = np.array(pd.DataFrame(spx1[:]).rolling(22).mean()[22-1:-1])

In [ ]:
# regress y over rv1 + rv5 + rv22
data = {'y': y, 'rv1': rv1, 'rv5': rv5, 'rv22': rv22}
fit_har = sm.ols('y ~ rv1 + rv5 + rv22', data=data).fit()

In [ ]:
print(fit_har.summary())

In [ ]:
fit_har.params

In [ ]:
# convert fitted values into pd.DataFrame
fitted = pd.DataFrame({'fitted': np.array(fit_har.fittedvalues)}, index=spx_rv[22:].index)
fitted.head()

In [ ]:
# log plot
plt.figure(figsize=(12, 6))
plt.plot(spx_rv, color='red', ls='dotted', label='Observed RV')
plt.plot(fitted, 'b', label='Forecasted RV')
plt.title('Observed and forecasted RV based on HAR model', fontsize=20, fontweight='bold')
plt.grid()
plt.legend();

### References
<br />

<div class="reflist" style="list-style-type: decimal;">

<ol>
  
  <li id="cite_note-Corsi"><span class="mw-cite-backlink"><b><a href="#cite_ref-Corsi">^</a></b></span>Fulvio Corsi, A simple approximate long-memory model of realized
volatility, <span>*Journal of Financial Econometrics*</span>
<span>**7**</span>(2) 174–196 (2009). </li>
  
  <li id="cite_note-ZMA"><span class="mw-cite-backlink"><b><a href="#cite_ref-ZMA">^</a></b></span>Lan Zhang, Per A. Mykland and Yacine Aït-Sahalia, A tale of two time scales: Determining intergrated volatility with noise high-frequency data, <span>*Journal of the American Statistical Association*</span>,
<span>**100**</span>(472), 1394–1411 (2005).
  </li>
  
  <li id="cite_note-Zhou"><span class="mw-cite-backlink"><b><a href="#cite_ref-Zhou">^</a></b></span>Bin Zhou, High-frequency data and volatility in foreign-exchange rates, <span>*Journal of Business & Economic Statistics*</span>, 
<span>**14**</span>(1), 45–52 (1996).
  </li>
</ol>
